### 1. Package setup

In [1]:
import scipy
import math
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
from curvefit.core.model import CurveModel
from curvefit import pv
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from curvefit.core.functions import log_erf, erf, expit, log_expit

### 2. Input data setup

In [2]:
# Model active cases
# Need to transform this data format to an amenable format for the IHME model
active_cases_us = 'timeseries_covid19_us_nhu_confirmed_cases_04102020.csv'
# Active cases as of 10 April from https://github.com/dishamakhija/covid19-india/blob/master/data/

In [3]:
def generalized_logistic(t, params) :
    alpha = params[0]
    beta  = params[1]
    p     = params[2]
    return p / ( 1.0 + np.exp( - alpha * ( t - beta ) ) )

### 3. Load input data

In [4]:
df_us = pd.read_csv(active_cases_us)
ny_cases = df_us.loc[df_us['Province/State'] == 'NY']

In [28]:
train_date_start = datetime.datetime(2020, 3, 4)
train_date_end = datetime.datetime(2020, 3, 16)
test_date_start = datetime.datetime(2020, 3, 17)
test_date_end = datetime.datetime(2020, 4, 9)

In [29]:
ny_cases

,Province/State,Country/Region,Lat,Long,02/28/2020,02/29/2020,03/01/2020,03/02/2020,03/03/2020,03/04/2020,...,03/31/2020,04/01/2020,04/02/2020,04/03/2020,04/04/2020,04/05/2020,04/06/2020,04/07/2020,04/08/2020,04/09/2020
37,NY,US,20,70,0.0,0.0,0.0,0.0,0.0,6.0,...,70820.0,77570.0,84947.0,93977.0,103226.0,109844.0,117323.0,124273.0,134726.0,145347.0


### 4. Prepare data in the IHME format

In [30]:
def fetch_columns(df, start_date, end_date, under_reporting_factor):
    """
    Helper function to fetch the columns from the active cases df
    Assumption: df contains only 1 row corresponding to the Province/Region 
    of interest
    """
    # The code for fetching the dates from the df is not clean
    # will clean it up later
    active_cases = []
    date = start_date
    delta = datetime.timedelta(days=1)
    count_df = pd.DataFrame(columns=["date", "active_count"])
    while date <= end_date:
        ##### NOTE - CHANGED DATE FORMAT FOR BLORE DATA
        num_active = df[date.strftime('%m/%d/%Y')].values[0]
        #num_active = df[date.strftime('%F')].values[0]
        count_df = count_df.append({
         "date": date,
         "active_count": num_active 
          }, ignore_index=True)
        date += delta
    
    count_df = count_df.sort_values(by='date', ascending=True)
    count_df.active_count = count_df.active_count*under_reporting_factor
    count_df['cumulative_count'] = count_df.active_count
    print(count_df)
    return count_df

def create_ihme_input(region, active_case_df, train_date_start, train_date_end, 
                      test_date_start, test_date_end,under_reporting_factor, col_covariate_vars,
                      social_distance):
    """
    active_case_df : Active caess for the Province/State under consideration (pd.Dataframe)
    train_date_start, train_date_end : Date range for train time frame
    test_date_start, test_date_end : Date range for test time frame
    """
    train_active_cases_df = fetch_columns(active_case_df, train_date_start, train_date_end, under_reporting_factor)
    test_active_cases_df = fetch_columns(active_case_df, test_date_start, test_date_end, under_reporting_factor)
    
    num_points = (train_active_cases_df.shape[0])
    
    assert(len(social_distance) == num_points)
    # As of now we assume all variates have same variance
    assert(len(col_covariate_vars) == num_points)
    
    num_train_days = (train_date_end - train_date_start).days + 1
    print(num_train_days)
    independent_var   = train_active_cases_df.index
    measurement_value = train_active_cases_df['cumulative_count']
    covariate_var     = col_covariate_vars
    print(covariate_var)
    social_distance   = [1 for i in range(num_train_days)]
    data_group        = num_train_days * [region]
    data_dict         = {
        'independent_var'   : independent_var   ,
        'measurement_value' : measurement_value ,
        'covariate_var'     : covariate_var     ,
        'social_distance'   : social_distance   ,
        'region'        : region        ,
    }
    train_df = pd.DataFrame(data_dict)
    print(train_df)
    return train_df, train_active_cases_df, test_active_cases_df

# Functions
# identity function
def identity_fun(x):
    return x
# link function used for alpha, p
def exp_fun(x):
    return np.exp(x)

def compute_mape(y_true, y_pred):
    mape = 0
    for i in range(len(y_pred)):
        if(not y_true[i] == 0):
            mape+= np.abs((y_true[i] - y_pred[i] + 0.) / y_true[i])
    mape = (100*mape)/len(y_pred)
    return mape

def rmse_error(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred)
            
    rmse = math.sqrt(rmse)
    return rmse

def rmsle_error(y_true, y_pred):
    rmsle = mean_squared_log_error(y_true, y_pred)
            
    rmsle = math.sqrt(rmsle)
    return rmsle



### 4. Model
### Model assumptions
- Independent variable - Day
- Measurement val - number of active cases
- Social distancing - 1.0  # TODO: Need to understand scale

In [31]:
region = 'NY'
num_train_days = (train_date_end - train_date_start).days + 1 # Account for the boundary case
print(num_train_days)
col_covariate_vars = num_train_days * [1.11]
social_distance = num_train_days * [10.0]
under_reporting_factor = 1
train_df, train_active_cases, test_active_cases = create_ihme_input(region, ny_cases,
                                                                   train_date_start, train_date_end,
                                                                   test_date_start, test_date_end, 
                                                                    under_reporting_factor,
                                                                   col_covariate_vars, social_distance)

13
         date  active_count  cumulative_count
0  2020-03-04           6.0               6.0
1  2020-03-05          22.0              22.0
2  2020-03-06          33.0              33.0
3  2020-03-07          76.0              76.0
4  2020-03-08         105.0             105.0
5  2020-03-09         142.0             142.0
6  2020-03-10         173.0             173.0
7  2020-03-11         216.0             216.0
8  2020-03-12         216.0             216.0
9  2020-03-13         421.0             421.0
10 2020-03-14         524.0             524.0
11 2020-03-15         729.0             729.0
12 2020-03-16         950.0             950.0
         date  active_count  cumulative_count
0  2020-03-17        1700.0            1700.0
1  2020-03-18        2382.0            2382.0
2  2020-03-19        4152.0            4152.0
3  2020-03-20        7102.0            7102.0
4  2020-03-21       10356.0           10356.0
5  2020-03-22       15168.0           15168.0
6  2020-03-23       20875.0    

In [32]:
# curve_model
num_params   = 3 # alpha, beta and p
num_fe       = 3 # fixed effect parameters
col_t        = 'independent_var'
col_obs      = 'measurement_value'
col_covs     = num_params *[['covariate_var']]
col_group    = 'region'
param_names  = ['alpha', 'beta', 'p']
link_fun     = [exp_fun, identity_fun, exp_fun ]
var_link_fun = num_fe * [identity_fun ]
fun          = erf

In [33]:
col_covs

[['covariate_var'], ['covariate_var'], ['covariate_var']]

In [34]:
# Initialize params
fe_init   = np.ones(num_fe)
re_init   = np.ones(num_fe)
fe_bounds = [[-np.inf, np.inf], [30, np.inf], [0, np.inf]] 
print(fe_bounds)
re_bounds = [[0.0, 0.0]] * num_fe

[[-inf, inf], [30, inf], [0, inf]]


In [42]:
var_a = [1e-1,0,1,10]
var_b = [1e-1,0,1,10]
var_c = [1e-1,0,1,10]

train_df['var_a'] = 0
train_df['var_b'] = 0
train_df['var_c'] = 0
all_metrics = []
for i in var_a:
    for j in var_b:
        for k in var_c:
            flat = {}
            train_df['var_a'] = i
            train_df['var_b'] = j
            train_df['var_c'] = k
            col_covs = [['var_a'],['var_b'],['var_c']]
            curve_model = CurveModel(
            train_df,
            col_t,
            col_obs,
            col_covs,
            col_group,
            param_names,
            link_fun,
            var_link_fun,
            fun
            )
            curve_model.fit_params(fe_init, re_init, fe_bounds, re_bounds)
            params_estimate = curve_model.params
            fe_estimate     = curve_model.result.x[: num_fe]
            flat['alpha'] = params_estimate[0][0]
            flat['beta'] = params_estimate[1][0]
            flat['p'] = params_estimate[2][0]
            flat['model'] = curve_model
            out_train = curve_model.predict(
            t=np.array(np.arange(0, 13)),
            group_name=region
            )
            
            if(( (np.max(out_train))!=np.inf) & ((np.isnan(out_train).any()==False))):
                flat['train_mape'] = compute_mape(train_active_cases['cumulative_count'], out_train)
                flat['train_rmsle'] = rmsle_error(train_active_cases['cumulative_count'], out_train)
            else:
                flat['train_mape'] = np.inf
                flat['train_rmsle'] = np.inf
            
            test_out = curve_model.predict(
            t=np.array(np.arange(14, 38)),
            group_name=region
            )
            
            
            if( ((np.max(test_out))!=np.inf) & ((np.isnan(test_out).any())==False)):
                flat['test_mape'] = compute_mape(test_active_cases['cumulative_count'], test_out)
                flat['test_rmsle'] = rmsle_error(test_active_cases['cumulative_count'], test_out)
            else:
                flat['test_mape'] = np.inf
                flat['test_rmsle'] = np.inf
            
            print(flat)
            all_metrics.append(flat)

{'alpha': 0.03136551084951289, 'beta': 158.52208809126108, 'p': 23731823084281.87, 'train_mape': 38.70154403353743, 'train_rmsle': 0.40406898987973144, 'test_mape': 98.67896447490152, 'test_rmsle': 0.7904086005147317}
{'alpha': 1.1051729497691727, 'beta': 3.0, 'p': 1.0, 'train_mape': 99.6449723743196, 'train_rmsle': 4.572045980656871, 'test_mape': 99.99218741524267, 'test_rmsle': 9.8551069889692}
{'alpha': 0.032196976374335454, 'beta': 150.75159878787437, 'p': 7215583488660.305, 'train_mape': 38.533895044836754, 'train_rmsle': 0.4028586587515609, 'test_mape': 95.15010600563842, 'test_rmsle': 0.7769773755734448}
{'alpha': 0.032524430828948125, 'beta': 147.8443466503819, 'p': 4614245557408.907, 'train_mape': 38.47438109409167, 'train_rmsle': 0.4024375361417872, 'test_mape': 93.72462268744665, 'test_rmsle': 0.7715128563778324}
{'alpha': 0.0923021898367291, 'beta': 0.0, 'p': 410.52040244370403, 'train_mape': 445.82237599956113, 'train_rmsle': 1.4135661935540889, 'test_mape': 96.85870681794

{'alpha': 22026.465794806718, 'beta': 3.003016850331323, 'p': 386.22044517039143, 'train_mape': 97.04586741485573, 'train_rmsle': 1.9653255210786493, 'test_mape': 96.98262003708642, 'test_rmsle': 4.683323857146812}
{'alpha': 22026.465794806718, 'beta': 3.0, 'p': 1.0, 'train_mape': 99.65467166120358, 'train_rmsle': 4.573426989619541, 'test_mape': 99.99218741524267, 'test_rmsle': 9.8551069889692}
{'alpha': 22019.69848617545, 'beta': 3.000027397926627, 'p': 386.222188331477, 'train_mape': 89.35389463237229, 'train_rmsle': 1.5527626163700907, 'test_mape': 96.98260641849271, 'test_rmsle': 4.683319525452156}
{'alpha': 22026.465794806718, 'beta': 3.0109328831833357, 'p': 386.22222235019547, 'train_mape': 97.04620778035331, 'train_rmsle': 1.9653259390694502, 'test_mape': 96.98260615271859, 'test_rmsle': 4.683319440917022}
{'alpha': 22026.465794806718, 'beta': 0.0, 'p': 294.693917000149, 'train_mape': 408.44497254311807, 'train_rmsle': 1.4418461406104512, 'test_mape': 97.69767879596206, 'test_r

In [44]:
df = pd.DataFrame(all_metrics)
sorted_df1 = df.sort_values(by='train_mape',ascending=True)
print(sorted_df1)

           alpha        beta             p   test_mape  test_rmsle  \
3       0.032524  147.844347  4.614246e+12   93.724623    0.771513   
2       0.032197  150.751599  7.215583e+12   95.150106    0.776977   
34      0.031360  158.597740  2.408898e+13   98.812660    0.790879   
0       0.031366  158.522088  2.373182e+13   98.678964    0.790409   
19      1.000000    8.843632  7.518797e+02   94.125876    4.047575   
18      1.000000    8.843632  7.518798e+02   94.125876    4.047575   
16      1.000000    8.843640  7.518807e+02   94.125868    4.047573   
50  22019.698486    3.000027  3.862222e+02   96.982606    4.683320   
32      4.064614    3.148417  3.862226e+02   96.982603    4.683318   
35      3.547748    3.169814  3.862192e+02   96.982629    4.683327   
48  22026.465795    3.003017  3.862204e+02   96.982620    4.683324   
51  22026.465795    3.010933  3.862222e+02   96.982606    4.683319   
53  22026.465795    0.000000  1.000000e+00   99.992187    9.855107   
37      2.718282    

In [40]:
print(sorted_df1.iloc[0]['model'].predict(
            t=np.array(np.arange(13, 25)),
            group_name=region
            ))
print(test_active_cases['cumulative_count'])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.02199238e-14 1.64362324e-04
 1.35914091e+00 2.71811747e+00 2.71828183e+00 2.71828183e+00
 2.71828183e+00 2.71828183e+00 2.71828183e+00 2.71828183e+00]
0       1700.0
1       2382.0
2       4152.0
3       7102.0
4      10356.0
5      15168.0
6      20875.0
7      25665.0
8      30811.0
9      37258.0
10     42590.0
11     49592.0
12     55941.0
13     62293.0
14     70820.0
15     77570.0
16     84947.0
17     93977.0
18    103226.0
19    109844.0
20    117323.0
21    124273.0
22    134726.0
23    145347.0
Name: cumulative_count, dtype: float64


In [41]:
print(out_train)
print(train_active_cases['cumulative_count'])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0       6.0
1      22.0
2      33.0
3      76.0
4     105.0
5     142.0
6     173.0
7     216.0
8     216.0
9     421.0
10    524.0
11    729.0
12    950.0
Name: cumulative_count, dtype: float64
